In [307]:
import os
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
from pymorphy2.tokenizers import simple_word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import pandas as pd
import re
from itertools import product

#### Создание словаря с предложениями, их разбором и источником

In [9]:
names = []
sents = []
lemma_seq = []
pos_seq = []

In [10]:
directory = 'C:\\Users\\Yana\\Desktop\\project\\corpus'
walk = [(x, y, z) for x, y, z in os.walk(directory)]

for root, dirs, files in walk:
    for f in files:
        f_path = os.path.join(root, f) 
        with open(f_path, 'r', encoding='utf-8') as f:
            text = f.read()
            
        name_num = text.find('\n')
        name = text[:name_num]
        
        sentences = sent_tokenize(text)
        sents.extend(sentences)
        
        for sent in sentences:
            tokens = simple_word_tokenize(sent)
            lemmas = []
            pos = []
            for word in tokens:
                if word[0].isalpha():
                    lemmas.append(str(morph.parse(word)[0].normal_form))
                    pos.append(str(morph.parse(word)[0].tag.POS))
            lemma_seq.append(' '.join(lemmas))
            pos_seq.append(' '.join(pos))
            names.append(name)

In [11]:
len(names), len(sents), len(lemma_seq), len(pos_seq)

(127857, 127857, 127857, 127857)

In [12]:
main_dict = {'source': names, 'sents': sents, 'lemmas': lemma_seq, 'pos':pos_seq}

#### Запись словаря в файл csv, чтобы потом больше не ждать
Файл используется только в коде с раширением .py

In [38]:
df = pd.DataFrame()
df['source'] = names
df['sents'] = sents
df['lemmas'] = lemma_seq
df['pos'] = pos_seq
df.head()

,source,sents,lemmas,pos
0,"""3 - глава 14"" Кикиморра","""3 - глава 14"" Кикиморра\n\n\n\n Глава 14.",глава кикиморра глава,NOUN NOUN NOUN
1,"""3 - глава 14"" Кикиморра","-- Азамат, я надеюсь, ты понимаешь, что это та...",азамат я надеяться ты понимать что это так ост...,NOUN NPRO VERB NPRO VERB CONJ PRCL CONJ INFN P...
2,"""3 - глава 14"" Кикиморра",-- Что именно?,что именно,CONJ PRCL
3,"""3 - глава 14"" Кикиморра",-- расеянно уточняет муж.,расеянный уточнять муж,ADJS VERB NOUN
4,"""3 - глава 14"" Кикиморра",-- Ситуацию с приютами.,ситуация с приют,NOUN PREP NOUN


In [70]:
df = df.dropna()

In [79]:
df2 = df
df2.to_csv('corpus.csv')

#### Функция собирает слова в предложении по модели токен|лемма|POS
Так можно будет искать все типы запросов в одной функции

In [14]:
def combine_sent(sent, lemma_seq, pos_seq):
    tokens = simple_word_tokenize(sent)
    norm_tokens = [token for token in tokens if token[0].isalpha()]
    lemmas = lemma_seq.split()
    pos = pos_seq.split()
    combined_list = []
    for i in range(len(pos)):
        combined_list.append(' ' + norm_tokens[i] + '|' + lemmas[i] + '|' + pos[i])
    return combined_list

#### Функция ищет предложения с словами из ввода

In [297]:
def search_func(main_dict, ans_dict, s_words):
    lemmas = main_dict['lemmas']
    for j in range(len(lemmas)):
        c_list = combine_sent(main_dict['sents'][j], main_dict['lemmas'][j], main_dict['pos'][j])
        for i in range(len(c_list)-len(s_words)+1):
            if s_words[0] in c_list[i]:
                if len(s_words) > 1 and s_words[1] in c_list[i+1]:
                    if len(s_words) == 3 and s_words[2] in c_list[i+2]:
                        ans_dict['sents'].append(main_dict['sents'][j])
                        ans_dict['source'].append(main_dict['source'][j])
                        key_words = [c.split('|')[0] for c in c_list[i:i+3]]
                        ans_dict['key_words'].append(' '.join(key_words))
                    elif len(s_words) == 2:
                        ans_dict['sents'].append(main_dict['sents'][j])
                        ans_dict['source'].append(main_dict['source'][j])
                        key_words = [c.split('|')[0] for c in c_list[i:i+2]]
                        ans_dict['key_words'].append(' '.join(key_words))
                elif len(s_words) == 1:
                    ans_dict['sents'].append(main_dict['sents'][j])
                    ans_dict['source'].append(main_dict['source'][j])
                    key_words = c_list[i].split('|')[0]
                    ans_dict['key_words'].append(key_words)
    return ans_dict

#### Функция создает все возможные комбинации слов
Нужно, если у словоформы есть несколько лемм.

In [222]:
def make_combos(words):
    word_combos = []
    if len(words) == 3:
        combos = list(product(words[0], words[1], words[2]))
        for c in combos:
            word_combos.append('.'.join(c))
    elif len(words) == 2:
        combos = list(product(words[0], words[1]))
        for c in combos:
            word_combos.append('.'.join(c))
    else:
        word_combos = words[0]
    return word_combos

#### Функция определяет тип слов из ввода и соединяет все функции выше

In [300]:
def mixed_search_upd(seq, main_dict):
    ans_dict = {'source':[], 'sents':[], 'key_words': []}
    
    s_words = seq.split(' ')
    for i in range(len(s_words)):
        if '"' in s_words[i]:
            s_words[i] = [' ' + s_words[i].replace('"', '') + '|']
        elif '+' in s_words[i]:
            parts = s_words[i].split('+')
            s_words[i] = [parts[0] + '|' + parts[1]]
        elif not s_words[i].isupper():
            normal_forms = []
            for m in morph.parse(s_words[i]):
                if m.normal_form not in normal_forms:
                    normal_forms.append(m.normal_form)
            if len(normal_forms) < 2:
                s_words[i] = ['|' + s_words[i].lower() + '|']
            else:
                s_words[i] = ['|' + word + '|' for word in normal_forms]
        else:
            s_words[i] = [s_words[i]]
    combos = make_combos(s_words)
    
    ans = []
    for combo in combos:
        ans_dict = search_func(main_dict, ans_dict, combo.split('.'))
        ans.append(ans_dict)
        
    return ans_dict

#### Парочка случайных примеров
Посмотреть более красивую выдачу можно в веб приложении. Ссылка в репозитории или в таблице.

In [308]:
ans_dict = mixed_search_upd('ADJ проект', main_dict)
df = pd.DataFrame(ans_dict)
df.head(10)

,source,sents,key_words
0,"""3 - глава 19"" Кикиморра","Кстати, я ей тут один проект предложила, она с...",один проект
1,"""35. О ходьбе по кругу"" Пивницкая Елена","Точно, ""химера"" же секретный проект!",секретный проект
2,"""43. О цене жизни и сделках с совестью"" Пивниц...","Там свои неписаные законы, зоны влияния и свои...",амбициозные проекты
3,"""Фантастическая политика и экономика"" Карнишин...",Что за новый проект?,новый проект
4,"""Когорта"" Зеленков Василий Вадимович","Без изысков, обычный имперский проект, с четко...",имперский проект
5,"""Когорта"" Зеленков Василий Вадимович","Она появилась на Альфии, когда я осваивался с ...",новым проектом
6,"""Когорта"" Зеленков Василий Вадимович","Я о них узнал днем позже, чем мой лорд; мы тог...",определенный проект
7,"""Когорта"" Зеленков Василий Вадимович",Над каким проектом он работал перед разговором...,каким проектом
8,"""Обитаемая планета"" Измайлова Кира","-- Представляю, это самое нудное, - отмахнулся...",индивидуальные проекты
9,"""Обитаемая планета"" Измайлова Кира","Оно пламенело -- то ли от стыда, то ли от горд...",свой проект


In [303]:
ans_dict = mixed_search_upd('у NPRO "проблемы"', main_dict)

In [304]:
df = pd.DataFrame(ans_dict)
df.head(10)

,source,sents,key_words
0,"""Chrome Shelled Regios Хромиованный региос том...","А теперь, когда она видит, что у тебя проблемы...",у тебя проблемы
1,"""И.О. капитана. Главы 18 - 25"" Сыромятникова И...","- У тебя проблемы, Рэм?",У тебя проблемы
2,"""Сердце на ладони. Книга вторая. Сирена и Вас...",- У нас проблемы.,У нас проблемы
